In [13]:
import datetime

import tensorflow as tf
import tensorflow_addons as tfa

from smot.jupyter import model_reports
from smot.problems.mnist import mnist_lib

In [14]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [15]:
(x_train, y_train), (x_test, y_test) = mnist_lib.load_mnist_data_28x28x1()

In [16]:
model = mnist_lib.build_ijcal2011_mnist_model(
    optimizer=tfa.optimizers.Lookahead(tf.keras.optimizers.Adam()),
    loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.1),
)

# Print the model summary.
model.summary()

batch_size = 512

datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=10,
    shear_range=0.1,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.2,
    validation_split=0.2,
)
datagen.fit(x_train)

training_generator = datagen.flow(
    x_train,
    y_train,
    subset="training",
    batch_size=batch_size,
)
validation_generator = datagen.flow(
    x_train,
    y_train,
    subset="validation",
    batch_size=batch_size,
)

Model: "LeNet5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 28, 28, 20)        520       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 14, 14, 20)       0         
 2D)                                                             
                                                                 
 conv2d_9 (Conv2D)           (None, 14, 14, 40)        20040     
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 7, 7, 40)         0         
 2D)                                                             
                                                                 
 conv2d_10 (Conv2D)          (None, 7, 7, 60)          60060     
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 3, 3, 60)         0    

In [17]:
%tensorboard --logdir logs/fit

Launching TensorBoard...

In [18]:
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=log_dir, histogram_freq=1, profile_batch="500,520"
)

history = model.fit(
    training_generator,
    validation_data=validation_generator,
    validation_freq=10,
    epochs=200,
    verbose=1,
    use_multiprocessing=True,
    workers=24,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(monitor="loss", patience=10),
        tensorboard_callback,
    ],
)

# Evaluate the model with the test data.
test_loss, test_accuracy = model_reports.model_fit_report(
    model=model,
    history=history,
    test_data=(x_test, y_test),
)

2021-12-20 01:36:29.021100: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2021-12-20 01:36:29.021136: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.
2021-12-20 01:36:29.465140: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.
2021-12-20 01:36:29.465344: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed


Epoch 1/200
94/94 [==============================] - 4s 24ms/step - loss: 1.2326 - accuracy: 0.7149
Epoch 2/200
94/94 [==============================] - 4s 25ms/step - loss: 0.7076 - accuracy: 0.9373
Epoch 3/200
94/94 [==============================] - 4s 25ms/step - loss: 0.6454 - accuracy: 0.9580
Epoch 4/200
94/94 [==============================] - 4s 24ms/step - loss: 0.6187 - accuracy: 0.9652
Epoch 5/200
94/94 [==============================] - 4s 25ms/step - loss: 0.6034 - accuracy: 0.9702
Epoch 6/200
27/94 [=======>......................] - ETA: 0s - loss: 0.5936 - accuracy: 0.9737

2021-12-20 01:36:52.368393: I tensorflow/core/profiler/lib/profiler_session.cc:110] Profiler session initializing.
2021-12-20 01:36:52.368436: I tensorflow/core/profiler/lib/profiler_session.cc:125] Profiler session started.


47/94 [==============>...............] - ETA: 1s - loss: 0.5934 - accuracy: 0.9739

2021-12-20 01:36:53.268912: I tensorflow/core/profiler/lib/profiler_session.cc:67] Profiler session collecting data.
2021-12-20 01:36:53.269454: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1764] CUPTI activity buffer flushed
2021-12-20 01:36:53.336454: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:526]  GpuTracer has collected 6654 callback api events and 6625 activity events. 
2021-12-20 01:36:53.407666: I tensorflow/core/profiler/lib/profiler_session.cc:143] Profiler session tear down.


50/94 [==============>...............] - ETA: 1s - loss: 0.5937 - accuracy: 0.9739

2021-12-20 01:36:53.517186: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/20211220-013629/plugins/profile/2021_12_20_01_36_53

2021-12-20 01:36:53.601293: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for trace.json.gz to logs/fit/20211220-013629/plugins/profile/2021_12_20_01_36_53/heatlamp.trace.json.gz
2021-12-20 01:36:53.684943: I tensorflow/core/profiler/rpc/client/save_profile.cc:136] Creating directory: logs/fit/20211220-013629/plugins/profile/2021_12_20_01_36_53

2021-12-20 01:36:53.690127: I tensorflow/core/profiler/rpc/client/save_profile.cc:142] Dumped gzipped tool data for memory_profile.json.gz to logs/fit/20211220-013629/plugins/profile/2021_12_20_01_36_53/heatlamp.memory_profile.json.gz
2021-12-20 01:36:53.691694: I tensorflow/core/profiler/rpc/client/capture_profile.cc:251] Creating directory: logs/fit/20211220-013629/plugins/profile/2021_12_20_01_36_53
Dumped tool data for xplane.pb to logs/

94/94 [==============================] - 5s 37ms/step - loss: 0.5908 - accuracy: 0.9745
Epoch 7/200
94/94 [==============================] - 4s 25ms/step - loss: 0.5830 - accuracy: 0.9768
Epoch 8/200
94/94 [==============================] - 4s 25ms/step - loss: 0.5761 - accuracy: 0.9791
Epoch 9/200
94/94 [==============================] - 4s 27ms/step - loss: 0.5702 - accuracy: 0.9805
Epoch 10/200
94/94 [==============================] - 6s 49ms/step - loss: 0.5670 - accuracy: 0.9814 - val_loss: 0.5714 - val_accuracy: 0.9797
Epoch 11/200
94/94 [==============================] - 4s 25ms/step - loss: 0.5624 - accuracy: 0.9830
Epoch 12/200
94/94 [==============================] - 4s 26ms/step - loss: 0.5583 - accuracy: 0.9840
Epoch 13/200
94/94 [==============================] - 4s 25ms/step - loss: 0.5552 - accuracy: 0.9854
Epoch 14/200
94/94 [==============================] - 4s 25ms/step - loss: 0.5536 - accuracy: 0.9860
Epoch 15/200
94/94 [==============================] - 4s 26ms/ste

KeyboardInterrupt: 